# Notebook for KFOLD training based on Walkie's code

In [1]:
import numpy as np
import os
import time
import argparse
import random
import sys
import tensorflow as tf
print(f"TensorFlow {tf.__version__}")



'''
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    print(f"Number of available GPUs : {len(gpus)}")
    tf.config.set_visible_devices(gpus[0],"GPU")
    tf.config.experimental.set_memory_growth(gpus[0],True)
else:
    print("No GPU available, using CPU !!!")    
    
# To disable GPU use
tf.config.set_visible_devices([], 'GPU')
'''
    
# with tf.device('CPU: 0'):
# with tf.device('GPU: 0'): 

#tf.device('CPU: 0')


# Define model hyperparameters 
seed = 175   # random seed
kfolds = 5   # number o folds
nfeat=3      # number of constituents features
batch=256    # batch size
nepochs=300   # number of epochs
#lr=0.0005    # learning rate
lr=0.0002    # learning rate
patience=10  # patience 

# Set numer of constituents and bitwidth
nconstit=16   # number of constituents
nbits=8      # QKeras bitwidth

# Define architecture name
arch = "QGCN"

TensorFlow 2.8.0


## Define the model

In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input,
    Concatenate,
    Flatten,
    BatchNormalization,
    Activation,
    GlobalAveragePooling1D,
    AveragePooling1D,
    Reshape,
    UpSampling1D,
    Add,
)
from tensorflow.keras.optimizers import Adam
from qkeras import QActivation, QDense, QConv1D, QConv2D, quantized_bits
from qkeras.autoqkeras.utils import print_qmodel_summary

from sklearn.metrics import accuracy_score
from tensorflow_model_optimization.python.core.sparsity.keras import pruning_wrapper
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

#############################################################################


# Quantized bits

#qbits = quantized_bits(nbits,integ,alpha=1.0)
#qact = 'quantized_relu('+str(nbits)+',0)'

# Set QKeras quantizer and activation 
if nbits == 1:
    qbits = 'binary(alpha=1)'
elif nbits == 2:
    qbits = 'ternary(alpha=1)'
else:
    qbits = 'quantized_bits({},0,alpha=1)'.format(nbits)

qact = 'quantized_relu({},0)'.format(nbits)

# Set QKeras linear activation quantization for CONV1D layers to avoid AveragePooling overflow
conv_qbits = 'quantized_bits(15,7)'

#############################################################################

# Load the model definition
if (arch=="QMLP"):
    execfile('mlp.py',globals(),locals())
elif (arch=="QGCN"):
    execfile('gcn.py',globals(),locals())
else:
    print("UNKNOWN ARCH !!! ",arch)
    stop
      
#############################################################################


# Print
print("Trainign with max # of contituents = ", nconstit)
print("Number of node features = ", nfeat)
print("Quantization with nbits=",nbits)



# create the model
model = Model(inputs=inp, outputs=out)

# Define the optimizer ( minimization algorithm )
optim = Adam(learning_rate=lr)

# compile the model
model.compile(optimizer=optim, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

# Print model summary
model.summary()

# Save model weights before training to re-initialize weights in each folding
model.save_weights('model.h5')

Trainign with max # of contituents =  16
Number of node features =  3
Quantization with nbits= 8
Metal device set to: Apple M1


2023-10-23 05:49:53.777342: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-23 05:49:53.777668: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Trainign with max # of contituents =  16
Number of node features =  3
Quantization with nbits= 8
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inp (InputLayer)               [(None, 16, 3)]      0           []                               
                                                                                                  
 conv1D_2 (QConv1D)             (None, 16, 40)       160         ['inp[0][0]']                    
                                                                                                  
 activ_conv_1 (QActivation)     (None, 16, 40)       0           ['conv1D_2[0][0]']               
                                                                                                  
 avgpool_1 (GlobalAveragePoolin  (None, 40)          0           ['activ_conv_1[0][0]']         

In [ ]:
import util.data
import util.plots
import util.util
from util.terminal_colors import tcols

random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

nmax = nconstit
accuracy_keras = []



# Loop of training folds
with tf.device('CPU: 0'):   # disable M1 metal GPU 
  for i in range (kfolds):
  
    # Load weights initialization values before any training ( reset weights ) 
    model.load_weights('model.h5')
    
    print("train kfold num:", i)
    val_kfold = i 

    #test_kfold = args.TK
    train_kfolds = [kfold for kfold in range(kfolds) if kfold != val_kfold]

    
    fpath = f'./data_kfold/jets_{nmax}constituents_ptetaphi_nonorm'
    fnames_train = [f'jet_images_c{nmax}_minpt2.0_ptetaphi_nonorm_{train_kfold}'
                     for train_kfold in train_kfolds]
    fname_val = f'jet_images_c{nmax}_minpt2.0_ptetaphi_nonorm_{val_kfold}'

    data = util.data.Data.load_kfolds(fpath, fnames_train, fname_val)
    print (data.train_data.shape)


    X_train = data.train_data
    X_val = data.test_data
    X_test = data.test_data

    Y_train = data.train_target
    Y_val = data.test_target
    Y_test = data.test_target    
  

    # Nomalization
    interquantile_range_32 = [120, 0.27, 0.27]
    interquantile_range_16 = [166, 0.24, 0.24]
    interquantile_range_8  = [219, 0.20, 0.20]
    
    if nmax == 8:
        X_train = X_train / interquantile_range_8
        X_val   = X_val   / interquantile_range_8
        X_test  = X_test  / interquantile_range_8
    elif nmax == 16:
        X_train = X_train / interquantile_range_16
        X_val   = X_val   / interquantile_range_16
        X_test  = X_test  / interquantile_range_16
    elif nmax == 32:
        X_train = X_train / interquantile_range_32
        X_val   = X_val   / interquantile_range_32
        X_test  = X_test  / interquantile_range_32

    # Flatten data for MLP input
    if (arch=='QMLP'):        
        X_train = X_train.reshape(-1, NINPUT)
        X_val   = X_val.reshape(-1, NINPUT)
        X_test  = X_test.reshape(-1, NINPUT)
    
    
    print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

    print( f"number of G jets for training/validation: {np.sum(np.argmax(Y_train, axis=1) == 0)}/{np.sum(np.argmax(Y_val, axis=1) == 0)}")
    print( f"number of Q jets for training/validation: {np.sum(np.argmax(Y_train, axis=1) == 1)}/{np.sum(np.argmax(Y_val, axis=1) == 1)}")
    print( f"number of W jets for training/validation: {np.sum(np.argmax(Y_train, axis=1) == 2)}/{np.sum(np.argmax(Y_val, axis=1) == 2)}")
    print( f"number of Z jets for training/validation: {np.sum(np.argmax(Y_train, axis=1) == 3)}/{np.sum(np.argmax(Y_val, axis=1) == 3)}")
    print( f"number of T jets for training/validation: {np.sum(np.argmax(Y_train, axis=1) == 4)}/{np.sum(np.argmax(Y_val, axis=1) == 4)}")

    
    print("number of G jets for testing: %i" % np.sum(np.argmax(Y_test, axis=1) == 0))
    print("number of Q jets for testing: %i" % np.sum(np.argmax(Y_test, axis=1) == 1))
    print("number of W jets for testing: %i" % np.sum(np.argmax(Y_test, axis=1) == 2))
    print("number of Z jets for testing: %i" % np.sum(np.argmax(Y_test, axis=1) == 3))
    print("number of T jets for testing: %i" % np.sum(np.argmax(Y_test, axis=1) == 4))
    
#######################################################################################

    mname = "model_{}_nconst_{}_nbits_{}_kfold_{}".format(arch,nmax,nbits,val_kfold)

    if(i==0):
        outputdir = "model_{}_nconst{}_nbits{}_Seed{}_Kfold_{}".format(
            arch,
            nconstit,
            nbits,
            seed,
            time.strftime("%Y%m%d-%H%M%S"),
        )
        print("output dir: ", outputdir)
        os.mkdir(outputdir)

        # early stopping callback
        es = EarlyStopping(monitor="val_categorical_accuracy", patience=patience)
        # Learning rate scheduler
        ls = ReduceLROnPlateau(monitor="val_categorical_accuracy", factor=0.2, patience=patience)

    # Train classifier
    chkp = ModelCheckpoint(outputdir+"/"+mname+".h5",
            monitor="val_categorical_accuracy",
            verbose=1,
            save_best_only=True,
            save_weights_only=False,
            mode="auto",
            save_freq="epoch")
    history = model.fit(
        X_train,
        Y_train,
        epochs=nepochs,
        batch_size=batch,  # small batch
        verbose=1,
        callbacks=[es, ls, chkp],
        #validation_split=0.2,
        validation_data=(X_val, Y_val))

    # Retrieve the best model
    model = tf.keras.models.load_model(
        "{}/{}.h5".format(outputdir, mname),
        custom_objects={
            "QDense": QDense,
            "QActivation": QActivation,
            "QConv1D": QConv1D,
            "QConv2D": QConv2D,
            "quantized_bits": quantized_bits,
#            "NodeEdgeProjection": NodeEdgeProjection,
#            "PruneLowMagnitude": pruning_wrapper.PruneLowMagnitude,
        },
    )

    # Get predictions for the best model
    y_keras = model.predict(X_test)
    
    # Store best model predictions
    accuracy_keras.append(float(
        accuracy_score(np.argmax(Y_test, axis=1), np.argmax(y_keras, axis=1))
    ))


train kfold num: 0

----------------
Data loading complete:
File name: None
Training data size: 480,840
Test data size: 120,210
Number of constituents: 16
Number of features: 3
----------------

(480840, 16, 3)
(480840, 16, 3) (120210, 16, 3) (480840, 5) (120210, 5)
number of G jets for training/validation: 96168/24042
number of Q jets for training/validation: 96168/24042
number of W jets for training/validation: 96168/24042
number of Z jets for training/validation: 96168/24042
number of T jets for training/validation: 96168/24042
number of G jets for testing: 24042
number of Q jets for testing: 24042
number of W jets for testing: 24042
number of Z jets for testing: 24042
number of T jets for testing: 24042
output dir:  model_QGCN_nconst16_nbits8_Seed175_Kfold_20231023-054956
Epoch 1/300


2023-10-23 05:49:56.759103: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


   1/1879 [..............................] - ETA: 25:04 - loss: 1.6510 - categorical_accuracy: 0.1289

2023-10-23 05:49:57.406671: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1878/1879 [============================>.] - ETA: 0s - loss: 1.3477 - categorical_accuracy: 0.4500

2023-10-23 05:50:11.786895: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.



Epoch 1: val_categorical_accuracy improved from -inf to 0.52479, saving model to model_QGCN_nconst16_nbits8_Seed175_Kfold_20231023-054956/model_QGCN_nconst_16_nbits_8_kfold_0.h5
1879/1879 [==============================] - 17s 9ms/step - loss: 1.3477 - categorical_accuracy: 0.4500 - val_loss: 1.2033 - val_categorical_accuracy: 0.5248 - lr: 2.0000e-04
Epoch 2/300
1879/1879 [==============================] - ETA: 0s - loss: 1.1566 - categorical_accuracy: 0.5417
Epoch 2: val_categorical_accuracy improved from 0.52479 to 0.55314, saving model to model_QGCN_nconst16_nbits8_Seed175_Kfold_20231023-054956/model_QGCN_nconst_16_nbits_8_kfold_0.h5
1879/1879 [==============================] - 19s 10ms/step - loss: 1.1566 - categorical_accuracy: 0.5417 - val_loss: 1.1275 - val_categorical_accuracy: 0.5531 - lr: 2.0000e-04
Epoch 3/300
1875/1879 [============================>.] - ETA: 0s - loss: 1.1131 - categorical_accuracy: 0.5603
Epoch 3: val_categorical_accuracy improved from 0.55314 to 0.56591,

## Saves accuracy and errors for paper plots

In [ ]:
#        kfold_metrics = {
#            "fprs": [],
#            "aucs": [],
#            "fats": [],
#            "accs": [],
#            "loss": []
#        }

#plots_dir = outputdir
#
#roc_metrics = util.plots.roc_curves(plots_dir, y_keras, data.test_target)
#util.plots.dnn_output(plots_dir, y_keras)

accuracy_average = np.mean(np.array(accuracy_keras))
accuracy_errs = np.std(np.array(accuracy_keras))


accs = np.zeros(3)
accs[0] = accuracy_average
accs[2] = accuracy_errs

np.savetxt("{}/acc.txt".format(outputdir), accs, fmt="%.6f")
print("Keras:\n", accuracy_keras)

print("output dir: ", outputdir)
